In [1]:
from docplex.mp.model import Model
import numpy as np

In [2]:
B = [15, 15, 15]
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]

In [3]:
def run_GAP_model(A, B, C):
    mdl = Model(name='GAP')
    n=len(A)
    m=len(A[0])
    
    # x[i][j] = 1 if i is assigned to j

    x=mdl.binary_var_matrix(range(n), range(m), name='x ' )
    #mdl.x=x
   
    
    
    # sum j: x_ij <= 1 for all i
    for i in range(n) :
        mdl.add_constraint(mdl.sum(x[i,j] for j in range(m)) == 1)

    # sum i: a_ij * x_ij <= b[j] for all j
    for j in range(m):
        mdl.add_constraint(mdl.sum(A[i][j] * x[i,j] for i in range(n)) <= B[j])
        
    # max sum i,j: c_ij * x_ij
    mdl.obj= mdl.sum((C[i][j] * x[i,j] for i in range(n) for j in range(m))) 
    mdl.minimize(mdl.obj)

    
    #mdl.print_information()
    msol = mdl.solve()

    assert msol is not None, "model can't solve"
    mdl.print_solution()
    


In [4]:
run_GAP_model(A,B,C)

objective: 18
  "x _0_2"=1
  "x _1_2"=1
  "x _2_1"=1
  "x _3_1"=1
  "x _4_2"=1


In [5]:
def run_GAP_model_with_Lagrangian_relaxation_pi(A, B, C, pi):
    mdl = Model(name='GAP')
    n=len(A)
    m=len(A[0])
    
    # x[i][j] = 1 if i is assigned to j
    x=mdl.binary_var_matrix(range(n), range(m), name='x ' )
   
    
    # variables
    penalties = mdl.continuous_var_list(n, name='penalties')  # Lagrangian multipliers

    # Dualized constraints: sum j: x_ij <= 1 for all i
    for i in range(n) :
        mdl.add_constraint(penalties[i]==1-mdl.sum(x[i,j] for j in range(m)) )
       
    # sum i: a_ij * x_ij <= b[j] for all j
    for j in range(m):
        mdl.add_constraint(mdl.sum(A[i][j] * x[i,j] for i in range(n)) <= B[j])
    
    mdl.obj= mdl.sum((C[i][j] * x[i,j] for i in range(n) for j in range(m)))+ mdl.sum(pi[i]*penalties[i] for i in range(n))
    mdl.minimize(mdl.obj)
        

    msol = mdl.solve()
    mdl.print_solution()
    
    return(mdl.objective_value)

    

In [12]:
n=len(A)
pi_test=[6]*n

run_GAP_model_with_Lagrangian_relaxation_pi(A, B, C, pi_test)


objective: 18.000
  "x _0_2"=1
  "x _1_2"=1
  "x _2_1"=1
  "x _3_1"=1
  "x _4_2"=1


18.0

In [13]:

def run_GAP_model_with_Lagrangian_relaxation(A, B, C, max_iters=101):
    mdl = Model(name='GAP')
    n=len(A)
    m=len(A[0])
    
    # x[i][j] = 1 if i is assigned to j
    x=mdl.binary_var_matrix(range(n), range(m), name='x ' )
   
    
    # variables
    penalties = mdl.continuous_var_list(n, name='penalties')  # Lagrangian multipliers

    # Dualized constraints: sum j: x_ij <= 1 for all i
    for i in range(n) :
        mdl.add_constraint(penalties[i]==1-mdl.sum(x[i,j] for j in range(m)))
       
    # sum i: a_ij * x_ij <= b[j] for all j
    for j in range(m):
        mdl.add_constraint(mdl.sum(A[i][j] * x[i,j] for i in range(n)) <= B[j])
    
    # pi[i] = Lagrangian Multiplier for the set packing contraint i
    pi = [0.0] * n

    
    # Re-optimize until either we have run a certain number of iterations
# or complementary slackness conditions apply.

    for k in range(1, max_iters):

        mdl.obj= mdl.sum((C[i][j] * x[i,j] for i in range(n) for j in range(m)))+ mdl.sum(pi[i]*penalties[i] for i in range(n))
        mdl.minimize(mdl.obj)
        

        msol = mdl.solve()
        #mdl.print_solution()

        print('--- iteration', k, ' ---- \nobj =', mdl.objective_value, \
              'pi =', pi, 'penalties =', [p.solution_value for p in penalties], '\n -------')
        
        # Test for complementary slackness
        stop = True
        eps = 10e-3
        for  p in  penalties:
            if  abs(p.solution_value) > eps:
                stop = False
                break

        if stop:
            print( 'primal feasible & optimal')
            print(k, 'iterations')
            break
        else:
            s = 1.0 / k
            for i in range(n):
                pi[i] = pi[i] + s*(penalties[i].solution_value)

In [ ]:

def run_GAP_model_with_Lagrangian_relaxation2(A, B, C, max_iters=101):
    mdl = Model(name='GAP')
    n=len(A)
    m=len(A[0])
    
    # x[i][j] = 1 if i is assigned to j
    x=mdl.binary_var_matrix(range(n), range(m), name='x ' )
   
    
    # variables
    penalties = mdl.continuous_var_list(n, name='penalties')  # Lagrangian multipliers

    # Dualized constraints: sum j: x_ij <= 1 for all i
    for i in range(n) :
        mdl.add_constraint(penalties[i]==1-mdl.sum(x[i,j] for j in range(m)))
       
    # sum i: a_ij * x_ij <= b[j] for all j
    for j in range(m):
        mdl.add_constraint(mdl.sum(A[i][j] * x[i,j] for i in range(n)) <= B[j])
    
    # pi[i] = Lagrangian Multiplier for the set packing contraint i
    pi = [0.0] * n

    
    # Re-optimize until either we have run a certain number of iterations
    # or complementary slackness conditions apply.

    for k in range(1, max_iters):

        mdl.obj= mdl.sum((C[i][j] * x[i,j] for i in range(n) for j in range(m)))+ mdl.sum(pi[i]*penalties[i] for i in range(n))
        mdl.minimize(mdl.obj)
        

        msol = mdl.solve()
        #mdl.print_solution()

        print('--- iteration', k, ' ---- \nobj =', mdl.objective_value, \
              'pi =', pi, 'penalties =', [p.solution_value for p in penalties], '\n -------')
        
        # Test for complementary slackness
        stop = True
        eps = 10e-3
        for  p in  penalties:
            if  abs(p.solution_value) > eps:
                stop = False
                break

        if stop:
            print( 'primal feasible & optimal')
            print(k, 'iterations')
            break
        else:
            alpha = 1
            s = alpha*()
            for i in range(n):
                pi[i] = pi[i] + s*(penalties[i].solution_value)

In [11]:
run_GAP_model_with_Lagrangian_relaxation(A,B,C,500)
#run_GAP_model_with_Lagrangian_relaxation2(A,B,C,500)

--- iteration 1  ---- 
obj = 0.0 pi = [0.0, 0.0, 0.0, 0.0, 0.0] penalties = [1.0, 1.0, 1.0, 1.0, 1.0] 
 -------
--- iteration 2  ---- 
obj = 5.0 pi = [1.0, 1.0, 1.0, 1.0, 1.0] penalties = [1.0, 1.0, 1.0, 1.0, 1.0] 
 -------
--- iteration 3  ---- 
obj = 7.0 pi = [1.5, 1.5, 1.5, 1.5, 1.5] penalties = [0, 1.0, 1.0, 1.0, 1.0] 
 -------
--- iteration 4  ---- 
obj = 8.333333333333332 pi = [1.5, 1.8333333333333333, 1.8333333333333333, 1.8333333333333333, 1.8333333333333333] penalties = [0, 1.0, 1.0, 1.0, 1.0] 
 -------
--- iteration 5  ---- 
obj = 9.333333333333332 pi = [1.5, 2.083333333333333, 2.083333333333333, 2.083333333333333, 2.083333333333333] penalties = [0, 1.0, 1.0, 1.0, 1.0] 
 -------
--- iteration 6  ---- 
obj = 10.133333333333333 pi = [1.5, 2.283333333333333, 2.283333333333333, 2.283333333333333, 2.283333333333333] penalties = [0, 1.0, 1.0, 1.0, 1.0] 
 -------
--- iteration 7  ---- 
obj = 10.799999999999999 pi = [1.5, 2.4499999999999997, 2.4499999999999997, 2.4499999999999997, 2.

--- iteration 127  ---- 
obj = 17.288877654429978 pi = [1.5, 5.148460267288507, 3.2888776544299785, 3.0198773448773446, 5.148460267288507] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 128  ---- 
obj = 17.29675167017801 pi = [1.5, 5.148460267288507, 3.29675167017801, 3.0198773448773446, 5.148460267288507] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 129  ---- 
obj = 17.296920534577012 pi = [1.5, 5.148460267288507, 3.30456417017801, 3.0198773448773446, 5.148460267288507] penalties = [0, 1.0, 0, 0, 1.0] 
 -------
--- iteration 130  ---- 
obj = 17.30456417017801 pi = [1.5, 5.156212205273003, 3.30456417017801, 3.0198773448773446, 5.156212205273003] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 131  ---- 
obj = 17.312256477870317 pi = [1.5, 5.156212205273003, 3.3122564778703176, 3.0198773448773446, 5.156212205273003] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 132  ---- 
obj = 17.312424410546008 pi = [1.5, 5.156212205273003, 3.319890065656577, 3.0198773448

--- iteration 226  ---- 
obj = 17.67541611286149 pi = [1.5, 5.339997875256744, 3.6754161128614924, 3.0198773448773446, 5.339997875256744] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 227  ---- 
obj = 17.679840891622554 pi = [1.5, 5.339997875256744, 3.6798408916225545, 3.0198773448773446, 5.339997875256744] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 228  ---- 
obj = 17.679995750513488 pi = [1.5, 5.339997875256744, 3.6842461779661666, 3.0198773448773446, 5.339997875256744] penalties = [0, 1.0, 0, 0, 1.0] 
 -------
--- iteration 229  ---- 
obj = 17.684246177966166 pi = [1.5, 5.344383840169025, 3.6842461779661666, 3.0198773448773446, 5.344383840169025] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 230  ---- 
obj = 17.688612990193242 pi = [1.5, 5.344383840169025, 3.6886129901932407, 3.0198773448773446, 5.344383840169025] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 231  ---- 
obj = 17.68876768033805 pi = [1.5, 5.344383840169025, 3.6929608162801975, 3.019

--- iteration 326  ---- 
obj = 17.921586740545308 pi = [1.5, 5.46086912414788, 3.9215867405453073, 3.0198773448773446, 5.46086912414788] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 327  ---- 
obj = 17.92173824829576 pi = [1.5, 5.46086912414788, 3.924654225207884, 3.0198773448773446, 5.46086912414788] penalties = [0, 1.0, 0, 0, 1.0] 
 -------
--- iteration 328  ---- 
obj = 17.924654225207885 pi = [1.5, 5.463927228123415, 3.924654225207884, 3.0198773448773446, 5.463927228123415] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 329  ---- 
obj = 17.92770300569569 pi = [1.5, 5.463927228123415, 3.927703005695689, 3.0198773448773446, 5.463927228123415] penalties = [0, 0, 1.0, 0, 0] 
 -------
--- iteration 330  ---- 
obj = 17.92785445624683 pi = [1.5, 5.463927228123415, 3.9307425193735006, 3.0198773448773446, 5.463927228123415] penalties = [0, 1.0, 0, 0, 1.0] 
 -------
--- iteration 331  ---- 
obj = 17.9307425193735 pi = [1.5, 5.466957531153718, 3.9307425193735006, 3.01987734487